In [2]:
%pip install termcolor > /dev/null

Note: you may need to restart the kernel to use updated packages.


In [1]:
import logging

logging.basicConfig(level=logging.ERROR)

In [2]:
from typing import List

from langchain.docstore import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from termcolor import colored

In [3]:
USER_NAME = "Person A"  # The name you want to use when interviewing the agent.
LLM = ChatOpenAI(model = "gpt-4o-mini", max_tokens=1500)  # Can be any LLM you want.

In [4]:
from langchain_experimental.generative_agents import (
    GenerativeAgent,
    GenerativeAgentMemory,
)

In [5]:
import math

import faiss


def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    # This will differ depending on a few things:
    # - the distance / similarity metric used by the VectorStore
    # - the scale of your embeddings (OpenAI's are unit norm. Many others are not!)
    # This function converts the euclidean norm of normalized embeddings
    # (0 is most similar, sqrt(2) most dissimilar)
    # to a similarity function (0 to 1)
    return 1.0 - score / math.sqrt(2)


def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    embeddings_model = OpenAIEmbeddings()
    # Initialize the vectorstore as empty
    embedding_size = 1536
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(
        embeddings_model.embed_query,
        index,
        InMemoryDocstore({}),
        {},
        relevance_score_fn=relevance_score_fn,
    )
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, other_score_keys=["importance"], k=15
    )

In [ ]:
# Definindo memórias e agentes NPCs de RPG
tommies_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=3,
)

# NPC 1 - Tommie
tommie = GenerativeAgent(
    name="Tommie",
    age=35,
    traits="brave, impulsive, quick thinker",  # Características de RPG
    status="looking for the lost artifact",  # Contexto de aventura
    memory_retriever=create_new_memory_retriever(),
    llm=LLM,
    memory=tommies_memory,
)

eves_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=3,
)

# NPC 2 - Eve
eve = GenerativeAgent(
    name="Eve",
    age=120,
    traits="wise, mysterious, protective",  # Características de RPG
    status="guardian of the forest",  # Contexto de aventura
    llm=LLM,
    daily_summaries=[
        (
            "Eve has guarded the ancient forest for over a century and holds knowledge of the artifact Tommie seeks."
        )
    ],
    memory=eves_memory,
    verbose=False,
)


In [8]:
def run_conversation(agents: List[GenerativeAgent], initial_observation: str) -> None:
    """Runs a conversation between agents."""
    _, observation = agents[1].generate_reaction(initial_observation)
    print(observation)
    turns = 0
    while True:
        break_dialogue = False
        for agent in agents:
            stay_in_dialogue, observation = agent.generate_dialogue_response(
                observation
            )
            print(observation)
            # observation = f"{agent.name} said {reaction}"
            if not stay_in_dialogue:
                break_dialogue = True
        if break_dialogue:
            break
        turns += 1

In [11]:
# Lista de agentes
agents = [tommie, eve]

# Iniciando a conversa
run_conversation(
    agents,
    "Tommie said: Greetings, Eve. I've heard you are the keeper of ancient wisdom. I seek the location of the lost artifact. Can you guide me?",
)

Eve said "The path to the lost artifact is fraught with challenges, Tommie. Are you prepared for what lies ahead?"
Tommie said "I appreciate your concern, Eve. Challenges are just opportunities for growth, and I'm ready to face whatever comes our way. Let's find that artifact!"
Eve said "Your spirit is commendable, Tommie. Remember, courage must be paired with wisdom. The journey ahead will test your resolve and your understanding. Are you ready to hear the first steps you must take?"
Tommie said "I'm ready, Eve. I understand that courage is important, but so is being wise in our decisions. Please share the first steps we need to take. I'm eager to get started!"
Eve said "Very well, Tommie. The first step on your journey is to seek the Elder Oak, which resides at the heart of the forest. It is said to hold the whispers of the ancients and will guide you further. But heed my words: approach with respect and patience, for the Elder Oak will test your intentions. Are you prepared to set f

KeyboardInterrupt: 